## Get Tutorial Data

## Assembly

In [ ]:
 ./trinityrnaseq-Trinity-v2.4.0/util/Trinity --seqType fq --max_memory 50G --left TrinityNatureProtocolTutorial/reads.ALL.left.fq  --right TrinityNatureProtocolTutorial/reads.ALL.right.fq --CPU 6

<b>Tool In Galaxy But Wrapper to improve</b>

## Quality assessment of the assembly

### Trinity Transcriptome Contig Nx 


<b>Stats about the assembled transcriptome, Nx transcript sizes. Uses a trinity script that needs to be wrapped, or RNA quast (need to build a conda recipe but could be based on the Quast recipe)</b>

In [1]:
./trinityrnaseq-Trinity-v2.4.0/util/TrinityStats.pl Trinity_on_data_3_and_data_4__Assembled_Transcripts.fasta



################################
## Counts of transcripts, etc.
################################
Total trinity 'genes':	8655
Total trinity transcripts:	8890
Percent GC: 37.99

########################################
Stats based on ALL transcript contigs:
########################################

	Contig N10: 3492
	Contig N20: 2853
	Contig N30: 2392
	Contig N40: 2053
	Contig N50: 1770

	Median contig length: 860
	Average contig: 1145.35
	Total assembled bases: 10182201


#####################################################
## Stats based on ONLY LONGEST ISOFORM per 'GENE':
#####################################################

	Contig N10: 3477
	Contig N20: 2852
	Contig N30: 2392
	Contig N40: 2054
	Contig N50: 1766

	Median contig length: 852
	Average contig: 1138.84
	Total assembled bases: 9856665





	Total assembled bases: 9856665

## Rnaseq read representation


"Assembled transcripts might not always fully represent properly paired-end reads, as some transcripts may be fragmented or short and only one fragment read of a pair may align. Simply aligning reads to your transcriptome assembly using bowtie or STAR will only capture the properly paired reads. To assess the read composition of our assembly, we want to capture and count all reads that map to our assembled transcripts, including the properly paired and those that are not.

In order to comprehensively capture read alignments, we run the process below. Bowtie2 is used to align the reads to the transcriptome and then we count the number of proper pairs and improper or orphan read alignments."

<b> Bowtie In Galaxy : Do all step in one with the option : Use a Genome in History and Build Index </b>

In [ ]:
bowtie2-build Trinity_on_data_3_and_data_4__Assembled_Transcripts.fasta Trinity_on_data_3_and_data_4__Assembled_Transcripts.fasta

In [ ]:
bowtie2 -p 10 -q -x  Trinity_on_data_3_and_data_4__Assembled_Transcripts.fasta -1 TrinityNatureProtocolTutorial/reads.ALL.left.fq -2 TrinityNatureProtocolTutorial/reads.ALL.right.fq  2>&1 1> /dev/null | tee align_stats.txt

In [ ]:
bowtie2 -p 10 -q -x  Trinity_on_data_3_and_data_4__Assembled_Transcripts.fasta -1 TrinityNatureProtocolTutorial/reads.ALL.left.fq -2 TrinityNatureProtocolTutorial/reads.ALL.right.fq  2>&1 1> /dev/null | samtools view -bS - > mapped.bam | tee align_stats.txt 

<b>Samtools  in Galaxy </b>

In [ ]:
# sort the alignments by coordinate
samtools sort mapped.bam -o bowtie2.coordSorted.bam

# index the coordinate-sorted bam file
samtools index bowtie2.nameSorted.1.bam

# index the Trinity.fasta file
samtools faidx Trinity.fasta

# view the aligned reads along the Trinity assembly reference contigs.
# note, you can do this by using the various graphical menu options in IGV (load genome 'Trinity.fasta', load file 'bowtie2.coordSorted.bam'), or you can use the command-line tool like so:

igv.sh -g `pwd`/Trinity.fasta  `pwd`/bowtie2.coordSorted.bam

## Transcript quantification


In [ ]:
./trinityrnaseq-Trinity-v2.4.0/util/align_and_estimate_abundance.pl  --transcripts Trinity_on_data_3_and_data_4__Assembled_Transcripts.fasta --seqType fa --left TrinityNatureProtocolTutorial/reads.ALL.left.fq --right TrinityNatureProtocolTutorial/reads.ALL.right.fq --est_method kallisto --output_dir output_trinity_quant --trinity_mode --prep_reference  

<b>Replaced by Salmon (In galaxy)</b>

In [ ]:
salmon index -t transcripts.fasta -i transcripts_index --type quasi -k 31

In [ ]:
salmon quant -i transcripts_index -l A -1 TrinityNatureProtocolTutorial/reads.ALL.left.fq -2 TrinityNatureProtocolTutorial/reads.ALL.right.fq -o transcripts_quant

<b>Or replaced by Kallisto (In test but not main)</b>

In [ ]:
kallisto index -i transcripts.idx transcripts.fasta

In [ ]:
kallisto quant -i transcripts.idx -o output -b 100 TrinityNatureProtocolTutorial/reads.ALL.left.fq TrinityNatureProtocolTutorial/reads.ALL.right.fq

<b>Count matrix Generation : to wrap</b>

In [ ]:
./trinityrnaseq-Trinity-v2.4.0/util/abundance_estimates_to_matrix.pl --out_prefix kallisto  --est_method kallisto  output_trinity_quant/abundance.tsv

### Trinity Transcriptome Contig Nx and ExNy Statistics


<b>to Wrap </b> 

In [ ]:
./trinityrnaseq-Trinity-v2.4.0/util/misc/contig_ExN50_statistic.pl kallisto.counts.matrix Trinity_on_data_3_and_data_4__Assembled_Transcripts.fasta | tee ExN50.stats

To plot the ExN50